# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Pytorch)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PPT file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  
**The TA will set a config file as 'eval_birds.yml' when evaluating the code using 'hidden test dataset'. Thus, please make sure that your code can generate proper data to measure inception score and R-precision of 'hidden test dataset'.**

## 1. Load datasets
The Birds dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [1]:
import torch
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import os, nltk
import numpy as np

from miscc.config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz

# from utils.data_utils import CUBDataset
from utils.datasets import CUBDataset
from utils.trainer import condGANTrainer

%matplotlib inline

In [2]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

Using config:
{'BATCH_SIZE': 32,
 'B_VALIDATION': False,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': '',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': True,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 32,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'LOAD_PRE_TRAIN': False,
 'NUM_BATCH_FOR_TEST': 0,
 'PRE_TRAIN': {'BATCH_SIZE': 32,
               'B_NET_D': True,
               'DISCRIMINATOR_LR': 0.0002,
               'ENCODER_LR': 0.0005,
               'FLAG': True,
               'GENERATOR_LR': 0.0002,
               'MAX_EPOCH': 5,
               'NET_E': '',
               'NET_G': '',
               'RNN_GRAD_CLIP': 0.25,
               'SMOOTH': {'GAMMA1': 4.0,
     

In [3]:
# image_transform = None # transforms.Compose([])

# train_dataset = CUBDataset(cfg.DATA_DIR, transform=image_transform, split='train')
# test_dataset = CUBDataset(cfg.DATA_DIR, transform=image_transform, split='test')

# print('train data directory:\n{}'.format(train_dataset.split_dir))
# print('test data directory:\n{}\n'.format(test_dataset.split_dir))

# print('# of train filenames: {}'.format(len(train_dataset.filenames)))
# print('# of test filenames: {}\n'.format(len(test_dataset.filenames)))

# print('example of filename of train image:{}'.format(train_dataset.filenames[0]))
# print('example of filename of valid image:{}\n'.format(test_dataset.filenames[0]))

# print('example of caption and its ids:\n{}\n{}\n'.format(train_dataset.captions[0], train_dataset.captions_ids[0]))
# print('example of caption and its ids:\n{}\n{}\n'.format(test_dataset.captions[0], test_dataset.captions_ids[0]))

# print('# of train captions:{}'.format(np.asarray(train_dataset.captions).shape))
# print('# of test captions:{}\n'.format(np.asarray(test_dataset.captions).shape))

# print('# of train caption ids:{}'.format(np.asarray(train_dataset.captions_ids).shape))
# print('# of test caption ids:{}\n'.format(np.asarray(test_dataset.captions_ids).shape))


In [4]:
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.TRAIN.BATCH_SIZE,
#         drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.TEST.BATCH_SIZE,
#         drop_last=True, shuffle=False, num_workers=int(cfg.WORKERS))
# # Dataloader for generating random (wrong) 9 captions which should be obtained to measure R-precision
# dataloader_for_wrong_samples = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.TEST.BATCH_SIZE,
#         drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))

In [5]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from utils.loss import sent_loss, words_loss
# from utils.data_utils import prepare_data
from utils.datasets import prepare_data

from utils.model import RNN_ENCODER, CNN_ENCODER

import sys
import time
import random
import argparse
from PIL import Image

import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

# dir_path = (os.path.abspath(os.path.join(os.path.realpath(__file__), './.')))
dir_path = os.getcwd()
sys.path.append(dir_path)

UPDATE_INTERVAL = 200

def train(dataloader, cnn_model, rnn_model, batch_size,
          labels, optimizer, epoch, ixtoword, image_dir):
    cnn_model.train()
    rnn_model.train()
    s_total_loss0 = 0
    s_total_loss1 = 0
    w_total_loss0 = 0
    w_total_loss1 = 0
    count = (epoch + 1) * len(dataloader)
    for step, data in enumerate(dataloader, 0):
        rnn_model.zero_grad()
        cnn_model.zero_grad()

        imgs, captions, cap_lens, \
        class_ids, keys, _ = prepare_data(data)

        words_features, sent_code = cnn_model(imgs[-1])
        nef, att_sze = words_features.size(1), words_features.size(2)

        hidden = rnn_model.init_hidden(batch_size)

        words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

        w_loss0, w_loss1, attn_maps = words_loss(words_features, words_emb, labels,
                                                 cap_lens, class_ids, batch_size)
        w_total_loss0 += w_loss0.data
        w_total_loss1 += w_loss1.data
        loss = w_loss0 + w_loss1

        s_loss0, s_loss1 = \
            sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        loss += s_loss0 + s_loss1
        s_total_loss0 += s_loss0.data
        s_total_loss1 += s_loss1.data
        loss.backward()
        torch.nn.utils.clip_grad_norm_(rnn_model.parameters(),
                                       cfg.PRE_TRAIN.RNN_GRAD_CLIP)
        optimizer.step()

        if step % UPDATE_INTERVAL == 0:
            count = epoch * len(dataloader) + step

            s_cur_loss0 = s_total_loss0.item() / UPDATE_INTERVAL
            s_cur_loss1 = s_total_loss1.item() / UPDATE_INTERVAL

            w_cur_loss0 = w_total_loss0.item() / UPDATE_INTERVAL
            w_cur_loss1 = w_total_loss1.item() / UPDATE_INTERVAL

            s_total_loss0 = 0
            s_total_loss1 = 0
            w_total_loss0 = 0
            w_total_loss1 = 0
            img_set, _ = \
                build_super_images(imgs[-1].cpu(), captions,
                                   ixtoword, attn_maps, att_sze)
            if img_set is not None:
                im = Image.fromarray(img_set)
                fullpath = '%s/attention_maps%d.png' % (image_dir, step)
                im.save(fullpath)
    return count


def evaluate(dataloader, cnn_model, rnn_model, batch_size):
    cnn_model.eval()
    rnn_model.eval()
    s_total_loss = 0
    w_total_loss = 0
    for step, data in enumerate(dataloader, 0):
        real_imgs, captions, cap_lens, \
        class_ids, keys, _ = prepare_data(data)

        words_features, sent_code = cnn_model(real_imgs[-1])

        hidden = rnn_model.init_hidden(batch_size)
        words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

        w_loss0, w_loss1, attn = words_loss(words_features, words_emb, labels,
                                            cap_lens, class_ids, batch_size)
        w_total_loss += (w_loss0 + w_loss1).data

        s_loss0, s_loss1 = \
            sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        s_total_loss += (s_loss0 + s_loss1).data

        if step == 50:
            break

    s_cur_loss = s_total_loss.item() / step
    w_cur_loss = w_total_loss.item() / step

    return s_cur_loss, w_cur_loss


def build_models():
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    labels = Variable(torch.LongTensor(range(batch_size)))
    start_epoch = 0
    if cfg.CUDA:
        text_encoder = text_encoder.cuda()
        image_encoder = image_encoder.cuda()
        labels = labels.cuda()

    return text_encoder, image_encoder, labels, start_epoch


def gen_example(wordtoix, algo):
    '''generate images from example sentences'''
    from nltk.tokenize import RegexpTokenizer
    filepath = '%s/example_filenames.txt' % (cfg.DATA_DIR)
    data_dic = {}
    with open(filepath, "r") as f:
        filenames = f.read().decode('utf8').split('\n')
        for name in filenames:
            if len(name) == 0:
                continue
            filepath = '%s/%s.txt' % (cfg.DATA_DIR, name)
            with open(filepath, "r") as f:
                print('Load from:', name)
                sentences = f.read().decode('utf8').split('\n')
                # a list of indices for a sentence
                captions = []
                cap_lens = []
                for sent in sentences:
                    if len(sent) == 0:
                        continue
                    sent = sent.replace("\ufffd\ufffd", " ")
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(sent.lower())
                    if len(tokens) == 0:
                        print('sent', sent)
                        continue

                    rev = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0 and t in wordtoix:
                            rev.append(wordtoix[t])
                    captions.append(rev)
                    cap_lens.append(len(rev))
            max_len = np.max(cap_lens)

            sorted_indices = np.argsort(cap_lens)[::-1]
            cap_lens = np.asarray(cap_lens)
            cap_lens = cap_lens[sorted_indices]
            cap_array = np.zeros((len(captions), max_len), dtype='int64')
            for i in range(len(captions)):
                idx = sorted_indices[i]
                cap = captions[idx]
                c_len = len(cap)
                cap_array[i, :c_len] = cap
            key = name[(name.rfind('/') + 1):]
            data_dic[key] = [cap_array, cap_lens, sorted_indices]
    algo.gen_example(data_dic)

## 2. Define models and go to train/evaluate

In [6]:

if not cfg.TRAIN.FLAG:
    manualSeed = 100
else:
    manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)
if cfg.CUDA:
    torch.cuda.manual_seed_all(manualSeed)

##########################################################################

if cfg.LOAD_PRE_TRAIN == False:
    now = datetime.datetime.now(dateutil.tz.tzlocal())
    timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
    output_dir = './checkpoint/pretrain_%s_%s_%s' % \
                 (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

    model_dir = os.path.join(output_dir, 'Model')
    best_model_dir = './checkpoint'

    image_dir = os.path.join(output_dir, 'Image')
    mkdir_p(model_dir)
    mkdir_p(image_dir)

    torch.cuda.set_device(int(cfg.GPU_ID))
    cudnn.benchmark = True

    imsize = cfg.PRE_TREE.BASE_SIZE * (2 ** (cfg.PRE_TREE.BRANCH_NUM - 1))
    batch_size = cfg.PRE_TRAIN.BATCH_SIZE
    image_transform = transforms.Compose([
        transforms.Resize(int(imsize * 76 / 64)),
        transforms.RandomCrop(imsize),
        transforms.RandomHorizontalFlip()])
    dataset = CUBDataset(cfg.DATA_DIR, 'train',
                          base_size=cfg.PRE_TREE.BASE_SIZE,
                          transform=image_transform)

    print(dataset.n_words, dataset.embeddings_num)
    assert dataset
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, drop_last=True,
        shuffle=True, num_workers=int(cfg.PRE_WORKERS))

    # # validation data #
    dataset_val = CUBDataset(cfg.DATA_DIR, 'test',
                              base_size=cfg.PRE_TREE.BASE_SIZE,
                              transform=image_transform)
    dataloader_val = torch.utils.data.DataLoader(
        dataset_val, batch_size=batch_size, drop_last=True,
        shuffle=True, num_workers=int(cfg.PRE_WORKERS))

    # Pretrain ##############################################################
    text_encoder, image_encoder, labels, start_epoch = build_models()
    para = list(text_encoder.parameters())
    for v in image_encoder.parameters():
        if v.requires_grad:
            para.append(v)
    min_loss = float("inf")

    try:
        lr = cfg.PRE_TRAIN.ENCODER_LR
        for epoch in range(start_epoch, cfg.PRE_TRAIN.MAX_EPOCH):
            optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
            count = train(dataloader, image_encoder, text_encoder,
                          batch_size, labels, optimizer, epoch,
                          dataset.ixtoword, image_dir)
            print('-' * 89)
            if len(dataloader_val) > 0:
                s_loss, w_loss = evaluate(dataloader_val, image_encoder,
                                          text_encoder, batch_size)
                print('| end epoch {:3d} | valid loss '
                      '{:5.2f} {:5.2f} | lr {:.5f}|'
                      .format(epoch, s_loss, w_loss, lr))
            print('-' * 89)
            if lr > cfg.PRE_TRAIN.ENCODER_LR / 10.:
                lr *= 0.98

            if (epoch % cfg.PRE_TRAIN.SNAPSHOT_INTERVAL == 0 or
                    epoch == cfg.PRE_TRAIN.MAX_EPOCH):
                torch.save(image_encoder.state_dict(),
                           '%s/image_encoder%d.pth' % (model_dir, epoch))
                torch.save(text_encoder.state_dict(),
                           '%s/text_encoder%d.pth' % (model_dir, epoch))
                print('Save G/Ds models.')

            if min_loss > s_loss + w_loss:
                min_loss = s_loss + w_loss
                torch.save(image_encoder.state_dict(),
                           '%s/image_encoder.pth' % best_model_dir)
                torch.save(text_encoder.state_dict(),
                           '%s/text_encoder.pth' % best_model_dir)
                print('| end epoch {:3d} | minimum loss {:5.2f}|'
                      .format(epoch, min_loss))

    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')




Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle
5450 10
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle


/home/pil-kso/anaconda3/envs/deep-learning-final/lib/python2.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


-----------------------------------------------------------------------------------------
| end epoch   0 | valid loss  6.73  6.71 | lr 0.00050|
-----------------------------------------------------------------------------------------
Save G/Ds models.
| end epoch   0 | minimum loss 13.44|
-----------------------------------------------------------------------------------------
| end epoch   1 | valid loss  6.52  6.52 | lr 0.00049|
-----------------------------------------------------------------------------------------
| end epoch   1 | minimum loss 13.04|
-----------------------------------------------------------------------------------------
| end epoch   2 | valid loss  6.45  6.44 | lr 0.00048|
-----------------------------------------------------------------------------------------
| end epoch   2 | minimum loss 12.89|
-----------------------------------------------------------------------------------------
| end epoch   3 | valid loss  6.25  6.38 | lr 0.00047|
------------------

In [7]:
# TRAIN #########################################################################

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = './checkpoint/%s_%s_%s' % \
             (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    split_dir = 'test'
    
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM - 1))
image_transform = transforms.Compose([
    transforms.Resize(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = CUBDataset(cfg.DATA_DIR, split_dir,
                      base_size=cfg.TREE.BASE_SIZE,
                      transform=image_transform)
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=cfg.TRAIN.BATCH_SIZE,
    drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

dataset_val = CUBDataset(cfg.DATA_DIR, 'test',
                          base_size=cfg.PRE_TREE.BASE_SIZE,
                          transform=image_transform)
dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=cfg.TEST.BATCH_SIZE, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

dataloader_for_wrong_samples = torch.utils.data.DataLoader(dataset_val, batch_size=cfg.TEST.BATCH_SIZE,
        drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))

algo = condGANTrainer(output_dir, dataloader, dataloader_val, dataloader_for_wrong_samples, dataset.n_words, dataset.ixtoword)

if cfg.TRAIN.FLAG:
    algo.train()
else:
    algo.generate_inception_score_data()
    algo.generate_r_precision_data()
    

Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle
Load my best image encoder
Load my best text encoder


utils/trainer.py:39: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(m.weight.data, 1.0)
utils/trainer.py:34: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(m.weight.data, 1.0)


# of netsD 3
errD0: 0.76 errD1: 0.83 errD2: 0.64 
g_loss0: 6.38 g_loss1: 8.75 g_loss2: 13.72 w_loss: 17.27 s_loss: 15.63 kl_loss: 0.00 
errD0: 0.68 errD1: 0.68 errD2: 3.17 
g_loss0: 7.80 g_loss1: 12.90 g_loss2: 2.23 w_loss: 15.73 s_loss: 17.12 kl_loss: 0.00 
errD0: 0.77 errD1: 0.60 errD2: 0.82 
g_loss0: 8.06 g_loss1: 11.26 g_loss2: 3.79 w_loss: 17.14 s_loss: 15.30 kl_loss: 0.00 
errD0: 1.17 errD1: 0.75 errD2: 1.21 
g_loss0: 6.71 g_loss1: 7.78 g_loss2: 3.14 w_loss: 7.91 s_loss: 9.62 kl_loss: 0.00 
errD0: 0.83 errD1: 0.68 errD2: 0.62 
g_loss0: 12.89 g_loss1: 20.82 g_loss2: 9.05 w_loss: 6.44 s_loss: 8.23 kl_loss: 0.00 
errD0: 0.17 errD1: 0.19 errD2: 0.55 
g_loss0: 12.39 g_loss1: 11.02 g_loss2: 8.14 w_loss: 17.09 s_loss: 12.82 kl_loss: 0.00 
errD0: 0.18 errD1: 0.29 errD2: 0.66 
g_loss0: 8.95 g_loss1: 9.93 g_loss2: 13.60 w_loss: 15.85 s_loss: 14.08 kl_loss: 0.00 
[0/5][737]
                  Loss_D: 2.11 Loss_G: 47.56 Time: 363.48s
Save G/Ds models.
errD0: 0.32 errD1: 0.57 errD2: 2.69 
g_lo

In [8]:
# EVALUATION #########################################################################

cfg_from_file('cfg/eval_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = './checkpoint/%s_%s_%s' % \
             (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    split_dir = 'test'
    
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM - 1))
image_transform = transforms.Compose([
    transforms.Resize(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = CUBDataset(cfg.DATA_DIR, split_dir,
                      base_size=cfg.TREE.BASE_SIZE,
                      transform=image_transform)
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=cfg.TRAIN.BATCH_SIZE,
    drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

dataset_val = CUBDataset(cfg.DATA_DIR, 'test',
                          base_size=cfg.PRE_TREE.BASE_SIZE,
                          transform=image_transform)
dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=cfg.TEST.BATCH_SIZE, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

dataloader_for_wrong_samples = torch.utils.data.DataLoader(dataset_val, batch_size=cfg.TEST.BATCH_SIZE,
        drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))

algo = condGANTrainer(output_dir, dataloader, dataloader_val, dataloader_for_wrong_samples, dataset.n_words, dataset.ixtoword)

if cfg.TRAIN.FLAG:
    algo.train()
else:
    algo.generate_inception_score_data()
    algo.generate_r_precision_data()

Using config:
{'BATCH_SIZE': 12,
 'B_VALIDATION': False,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': '',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': True,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 32,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'LOAD_PRE_TRAIN': True,
 'NUM_BATCH_FOR_TEST': 0,
 'PRE_TRAIN': {'BATCH_SIZE': 32,
               'B_NET_D': True,
               'DISCRIMINATOR_LR': 0.0002,
               'ENCODER_LR': 0.0005,
               'FLAG': True,
               'GENERATOR_LR': 0.0002,
               'MAX_EPOCH': 5,
               'NET_E': '',
               'NET_G': '',
               'RNN_GRAD_CLIP': 0.25,
               'SMOOTH': {'GAMMA1': 4.0,
      

./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0045_2303_9.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0035_2611_8.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0012_2691_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0004_2345_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0030_2268_0.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0053_13391_0.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0026_795066_4.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0024_797529_3.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0094_1540_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0066_2693_3.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0006_1763_9.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0012_795074_6.png

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0036_796127_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0096_2634_0.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0005_2111_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0099_2560_6.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0019_1585_7.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0045_797536_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090_6.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0016_2225_7.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0027_1754_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0047_796064_4.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0034_795096_5.png
./evaluation/generated_images/

./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0060_1505_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0009_2616_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0010_2269_0.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0007_796138_9.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0051_1650_1.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0007_797521_2.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0004_795112_0.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0091_1728_6.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0017_2155_7.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0034_797497_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0076_417_8.png
./evaluation/generated_images/001.Black_f

./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0062_1767_5.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0053_13391_8.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0072_14197_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55_2.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0031_1588_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0064_2290_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0016_2225_8.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0012_1784_7.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0043_795067_7.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0135_2607_3.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0018_1613_4.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_

...

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0076_417_0.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0065_2310_0.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0054_795130_1.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0115_2279_5.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0077_1724_5.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0018_1613_3.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0053_1672_8.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0070_2325_0.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0056_1493_3.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0037_796120_9.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0006_1763_2.png
./evaluation/generated_images/008.Rhinoceros_Au

./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0005_1750_7.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0014_14167_2.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0139_2567_6.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0009_797539_1.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0135_2607_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0008_796083_6.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0016_796067_6.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0037_1560_1.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_3.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0082_1697_9.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0010_797545_7.png
./evaluation/generated_images/004.G

./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0100_1646_2.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0043_795067_2.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0037_14128_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0064_796101_3.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0041_2653_9.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55_4.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0050_796125_0.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0032_795068_7.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0085_92_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0041_2653_8.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0028_795094_4.png
./evaluation/generated_images/001.Black_footed_

./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0059_11596_3.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0049_797543_0.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0031_1588_1.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0092_1516_4.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0044_1731_7.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0046_795118_1.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0027_1754_4.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0006_797512_1.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0086_796062_6.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0044_797502_4.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0007_1615_2.png
./evaluation/generated_images/004.Groove_billed_A

./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0008_796083_9.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0051_1650_1.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Billed_Ani_0040_1715_7.png
./evaluation/generated_images/014.Indigo_Bunting/Indigo_Bunting_0001_12469_3.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0025_795087_2.png
./evaluation/generated_images/001.Black_footed_Albatross/Black_Footed_Albatross_0079_796122_4.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0017_2668_9.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0011_795109_9.png
./evaluation/generated_images/008.Rhinoceros_Auklet/Rhinoceros_Auklet_0007_797521_6.png
./evaluation/generated_images/009.Brewer_Blackbird/Brewer_Blackbird_0090_2658_5.png
./evaluation/generated_images/006.Least_Auklet/Least_Auklet_0026_795066_5.png
./evaluation/generated_images/004.Groove_billed_Ani/Groove_Bi

## 3. Measure Inception score and R-precision of given test dataset

After set the config file as 'eval_birds.yml' and run the 'algo.generate_inception_score_data()' and 'algo.generate_r_precision_data()', the synthesized images based on given captions and set of image and caption features should be saved inside a 'evaluation' folder, specifically in 'evaluation/generated_images/..' and as 'evaluation/r_precision.npz' respectively.

**Then, go to the 'evaluation' folder and run each 'inception_score.ipynb' and 'r_precision.ipynb' file in order to measure inception score and r-precision score.**